In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import iris
plt.rcParams['font.sans-serif']=['Arial']

In [2]:
Y = 5
lon = 720
lat = 360

In [3]:
res_lon = 360/lon
res_lat = 180/lat
sample_points = [('longitude', np.linspace(0+res_lon/2, 360-res_lon/2, lon)),
                 ('latitude',  np.linspace(-90+res_lat/2, 90-res_lat/2, lat))]

In [4]:
sample_points

[('longitude',
  array([2.5000e-01, 7.5000e-01, 1.2500e+00, 1.7500e+00, 2.2500e+00,
         2.7500e+00, 3.2500e+00, 3.7500e+00, 4.2500e+00, 4.7500e+00,
         5.2500e+00, 5.7500e+00, 6.2500e+00, 6.7500e+00, 7.2500e+00,
         7.7500e+00, 8.2500e+00, 8.7500e+00, 9.2500e+00, 9.7500e+00,
         1.0250e+01, 1.0750e+01, 1.1250e+01, 1.1750e+01, 1.2250e+01,
         1.2750e+01, 1.3250e+01, 1.3750e+01, 1.4250e+01, 1.4750e+01,
         1.5250e+01, 1.5750e+01, 1.6250e+01, 1.6750e+01, 1.7250e+01,
         1.7750e+01, 1.8250e+01, 1.8750e+01, 1.9250e+01, 1.9750e+01,
         2.0250e+01, 2.0750e+01, 2.1250e+01, 2.1750e+01, 2.2250e+01,
         2.2750e+01, 2.3250e+01, 2.3750e+01, 2.4250e+01, 2.4750e+01,
         2.5250e+01, 2.5750e+01, 2.6250e+01, 2.6750e+01, 2.7250e+01,
         2.7750e+01, 2.8250e+01, 2.8750e+01, 2.9250e+01, 2.9750e+01,
         3.0250e+01, 3.0750e+01, 3.1250e+01, 3.1750e+01, 3.2250e+01,
         3.2750e+01, 3.3250e+01, 3.3750e+01, 3.4250e+01, 3.4750e+01,
         3.5250e+01

In [5]:
t = Y*12 ## Y years correspond to 12Y months 

lon_range = np.linspace(0+res_lon/2, 360-res_lon/2, lon)
lat_range = np.linspace(-90+res_lat/2, 90-res_lat/2, lat) 
t_range   = np.linspace(1, t, t) 
lat_mesh, t_mesh, lon_mesh = np.meshgrid(lat_range, t_range, lon_range) 

t_grid   = t_mesh.reshape(t*lon*lat, 1)
lon_grid = lon_mesh.reshape(t*lon*lat, 1)
lat_grid = lat_mesh.reshape(t*lon*lat, 1)

dataset = pd.DataFrame(lon_grid, columns = {'longitude'})
dataset['latitude'] = lat_grid
dataset['t'] = t_grid
dataset['loc'] = dataset['longitude']*1000+dataset['latitude']
dataset

,longitude,latitude,t,loc
0,0.25,-89.75,1.0,160.25
1,0.75,-89.75,1.0,660.25
2,1.25,-89.75,1.0,1160.25
3,1.75,-89.75,1.0,1660.25
4,2.25,-89.75,1.0,2160.25
...,...,...,...,...
15551995,357.75,89.75,60.0,357839.75
15551996,358.25,89.75,60.0,358339.75
15551997,358.75,89.75,60.0,358839.75
15551998,359.25,89.75,60.0,359339.75


In [6]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/o3prod_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'o3prod')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o3prod = values.reshape(t*lon*lat,1)
dataset['o3prod'] = o3prod

In [7]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/o3loss_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'o3loss')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o3loss = values.reshape(t*lon*lat,1)
dataset['o3loss'] = o3loss

In [8]:
dataset_pt1 = dataset[dataset['longitude'] < 180]
dataset_pt2 = dataset[dataset['longitude'] >= 180]
dataset_pt2['longitude'] = dataset_pt2['longitude']-360
dataset_adj = dataset_pt1.append(dataset_pt2)
dataset_adj

,longitude,latitude,t,loc,o3prod,o3loss
0,0.25,-89.75,1.0,160.25,8.874727e-14,9.524117e-14
1,0.75,-89.75,1.0,660.25,8.873214e-14,9.523655e-14
2,1.25,-89.75,1.0,1160.25,8.873399e-14,9.524327e-14
3,1.75,-89.75,1.0,1660.25,8.874603e-14,9.525680e-14
4,2.25,-89.75,1.0,2160.25,8.875808e-14,9.527032e-14
...,...,...,...,...,...,...
15551995,-2.25,89.75,60.0,357839.75,3.649463e-16,1.918938e-16
15551996,-1.75,89.75,60.0,358339.75,3.846478e-16,1.967400e-16
15551997,-1.25,89.75,60.0,358839.75,4.043494e-16,2.015861e-16
15551998,-0.75,89.75,60.0,359339.75,4.359465e-16,2.069088e-16


In [ ]:
dataset_adj.to_csv('/Volumes/Database/Research/ozone-budget/UKESM1-0-LL_2010_2014.csv', index=False)

In [ ]:
dataset = pd.read_csv('/Volumes/Database/Research/ozone-budget/UKESM1-0-LL_2010_2014.csv')
dataset

,longitude,latitude,t,loc,o3prod,o3loss
0,0.25,-89.75,1.0,160.25,8.874727e-14,9.524117e-14
1,0.75,-89.75,1.0,660.25,8.873214e-14,9.523655e-14
2,1.25,-89.75,1.0,1160.25,8.873399e-14,9.524327e-14
3,1.75,-89.75,1.0,1660.25,8.874603e-14,9.525680e-14
4,2.25,-89.75,1.0,2160.25,8.875808e-14,9.527032e-14
...,...,...,...,...,...,...
15551995,-2.25,89.75,60.0,357839.75,3.649463e-16,1.918938e-16
15551996,-1.75,89.75,60.0,358339.75,3.846478e-16,1.967400e-16
15551997,-1.25,89.75,60.0,358839.75,4.043494e-16,2.015861e-16
15551998,-0.75,89.75,60.0,359339.75,4.359465e-16,2.069088e-16


In [12]:
dataset['o3prod_mole'] = dataset['o3prod']*6.022*10**17
dataset['o3loss_mole'] = dataset['o3loss']*6.022*10**17

In [13]:
o3net_map = dataset.groupby(['loc']).mean()
o3net_map

,longitude,latitude,t,o3prod,o3loss,o3prod_mole,o3loss_mole,o3net
loc,,,,,,,,
160.25,0.25,-89.75,30.5,2.463897e-14,3.095005e-14,14837.586833,18638.121870,-3800.535037
160.75,0.25,-89.25,30.5,2.490451e-14,3.136068e-14,14997.493136,18885.401326,-3887.908190
161.25,0.25,-88.75,30.5,2.517004e-14,3.177131e-14,15157.399168,19132.680721,-3975.281553
161.75,0.25,-88.25,30.5,2.543558e-14,3.218193e-14,15317.305501,19379.960266,-4062.654765
162.25,0.25,-87.75,30.5,2.591031e-14,3.299281e-14,15603.186111,19868.271844,-4265.085732
...,...,...,...,...,...,...,...,...
359837.75,-0.25,87.75,30.5,1.819803e-13,1.321836e-13,109588.534886,79600.979177,29987.555709
359838.25,-0.25,88.25,30.5,1.973328e-13,1.318272e-13,118833.823628,79386.355388,39447.468239
359838.75,-0.25,88.75,30.5,2.121464e-13,1.319541e-13,127754.579954,79462.745667,48291.834287


In [ ]:
o3net_map.to_csv('/Users/csuen/Documents/GitHub/ozone-radical/Fig. 3/Fig. 3 ozone budget geomapping.csv', index=False)

#### Statistics of terrestrial ozone chemical budget

In [ ]:
dataset = pd.read_csv('/Users/csuen/radicals/UKESM1-0-LL_2010_2014.csv')

land = pd.read_csv('/Users/csuen/Documents/GitHub/ozone-budget/global_grid_2x2_label_continent.csv')
dataset_land = dataset.merge(land, how='left', left_on=['longitude','latitude'], right_on=['longitude','latitude'])
dataset_land = dataset_land[dataset_land['land_global']==1]

In [16]:
map = dataset_land.groupby(['loc_x']).mean()

In [17]:
map['o3prod_mole_stacked'].mean()

4934663.054742577

In [19]:
map['o3prod_mole_stacked'].std()

5306477.679726478

In [20]:
map['o3prod_mole_stacked'].median()

2945092.6126391385

In [21]:
np.nanpercentile(map['o3prod_mole_stacked'],25)

790299.3010800732

In [22]:
np.nanpercentile(map['o3prod_mole_stacked'],75)

7470303.398270063

In [24]:
map['o3prod_mole_stacked'].max()

33627057.3163171

In [25]:
map['o3prod_mole_stacked'].min()

78083.09860710982

In [26]:
np.nanpercentile(map['o3prod_mole_stacked'],5)

182886.50314983985

In [27]:
np.nanpercentile(map['o3prod_mole_stacked'],95)

16202433.648521382

In [18]:
map['o3loss_mole_stacked'].mean()

835226.2450608012

In [28]:
map['o3loss_mole_stacked'].std()

753411.8583575211

In [29]:
map['o3loss_mole_stacked'].median()

584650.8649564637

In [30]:
np.nanpercentile(map['o3loss_mole_stacked'],25)

175280.41593605865

In [31]:
np.nanpercentile(map['o3loss_mole_stacked'],75)

1396014.9054213737

In [32]:
np.nanpercentile(map['o3loss_mole_stacked'],5)

93199.824945032

In [33]:
np.nanpercentile(map['o3loss_mole_stacked'],95)

2202623.711114734

In [34]:
map['o3loss_mole_stacked'].max()

3943688.905442245

In [35]:
map['o3loss_mole_stacked'].min()

65115.3642247982